# EasyOCR 값 분리

## 테스트 원본 코드

In [2]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 5개)
file_paths = file_paths[:5]
file_count = 5


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    '''
    # 문자 인식 결과값 출력
    print("")
    print("<OCR 인식 결과>")
    for i in result:
        print(i[1], end=" || ")
    print("")
    print("[인식한 문자] : 총 "+str(len(result))+"개")
    '''
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        '''
        # 라벨링 데이터의 문자 값만 출력
        print("")
        print("<라벨링 데이터>")
        for k in json_test:
            print(k['annotation.text'], end=" || ")
        print("")
        print("[라벨링 문자] : 총 "+str(len(json_test))+"개")
        '''
    
        # 인식 결과값과 라벨링 데이터 값 추출
        result_texts = {i[1] for i in result}
        json_test_texts = {k['annotation.text'] for k in json_test}
        
        result_list = []
        json_test_list = []
        matching_texts = []
        
        # 인식한 문자를 한 글자씩 분리
        for text_result in sorted(result_texts):
            result_list.extend(list(text_result))
        for text_json in sorted(json_test_texts):
            json_test_list.extend(list(text_json))
            
        '''
        # 분리 결과 출력
        print("")
        print("<인식한 문자 분리 결과>")
        print(result_list)
        print("[분리한 인식 문자] : 총 "+str(len(result_list))+"개")
        print("")
        print("<라벨링 문자 분리 결과>")
        print(json_test_list)
        print("[분리한 라벨링 문자] : 총 "+str(len(json_test_list))+"개")
        '''
        
        # 한 글자씩 비교하여 일치하는 값을 matching_texts 리스트에 저장
        for item in result_list:
            if item in json_test_list:
                matching_texts.append(item)
        
        '''
        # 일치 문자 출력
        print("")
        print("<일치 문자 결과>")
        print(matching_texts)
        print("[일치한 문자] : 총 "+str(len(matching_texts))+"개")
        print("")
        '''
    
        # 정확도 계산
        percentage = (len(matching_texts) / len(result_list)) * 100
    
        # 정확도, 경과 시간 출력
        if matching_texts:
            print("정확도 : {:.2f}%".format(percentage))
            print("경과 시간 : {:.2f}초".format(elapsed_time))
            total_accuracy += percentage
        else:
            print("정확한 인식 값이 없습니다.")
            
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))
print("평균 정확도 : {:.2f}%".format(average_accuracy))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중
정확도 : 96.44%
경과 시간 : 2.71초

파일명 : 5350109-2001-0001-0002.jpg
인식 실행 중
정확도 : 84.35%
경과 시간 : 1.07초

파일명 : 5350109-2001-0001-0004.jpg
인식 실행 중
정확도 : 96.22%
경과 시간 : 1.97초

파일명 : 5350109-2001-0001-0005.jpg
인식 실행 중
정확도 : 95.25%
경과 시간 : 2.13초

파일명 : 5350109-2001-0001-0006.jpg
인식 실행 중
정확도 : 81.13%
경과 시간 : 0.96초

총 데이터 수 : 5개
총 소요 시간 : 0분 8초
평균 인식 시간 : 1.77초
평균 정확도 : 90.68%


## 텍스트 인식 영역만 출력

readtext 후 슬라이싱

In [19]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 1개)
file_paths = file_paths[:1]
file_count = 1


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    
    # 문자 인식 결과 출력
    print("")
    print("<OCR 인식 결과>")
    print(result)
    # 좌표 값 출력
    for i in result:
        print(i[0])
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        # 라벨링 데이터의 좌표 값만 출력
        print("")
        print("<라벨링 데이터>")
        # bbox 좌표 값 출력
        for k in json_test:
            print(k['annotation.bbox'])
    
        # 경과 시간 출력
        print("")
        print("경과 시간 : {:.2f}초".format(elapsed_time))
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중

<OCR 인식 결과>
[([[952, 319], [1126, 319], [1126, 382], [952, 382]], '"김해틀', 0.6891791224479675), ([[1138, 316], [1340, 316], [1340, 385], [1138, 385]], '아름답게', 0.9564188718795776), ([[1352, 316], [1506, 316], [1506, 382], [1352, 382]], '시민흘', 0.21049218836622702), ([[1518, 316], [1742, 316], [1742, 385], [1518, 385]], '행복하게"', 0.9996494662489984), ([[954, 385], [1035, 385], [1035, 466], [954, 466]], '김', 0.9998815094763955), ([[1319, 381], [1408, 381], [1408, 465], [1319, 465]], '해', 0.9999432571831335), ([[367, 481], [420, 481], [420, 539], [367, 539]], '우', 0.9999682905803553), ([[433, 480], [676, 480], [676, 540], [433, 540]], '621-010/', 0.8502205921099988), ([[713, 477], [832, 477], [832, 543], [713, 543]], '경남', 0.9630026380685481), ([[833, 477], [993, 477], [993, 543], [833, 543]], '김해시', 0.9941473818466108), ([[1010, 477], [1168, 477], [1168, 543], [1010, 543]], '부원동', 0.9962835228060669), ([[1180, 480], [1284, 480], [1284, 540], [1180, 5

## 문자 값만 출력

readtext 후 슬라이싱

In [18]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 1개)
file_paths = file_paths[:1]
file_count = 1


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    # 문자 인식 결과 출력
    print("")
    print("<OCR 인식 결과>")
    print(result)
    # 인식한 문자 출력
    for i in result:
        print(i[1])
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        # 라벨링 데이터의 문자 값만 출력
        print("")
        print("<라벨링 데이터>")
        # 인식한 문자 출력
        for k in json_test:
            print(k['annotation.text'])
            
        print("경과 시간 : {:.2f}초".format(elapsed_time))     
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중

<OCR 인식 결과>
[([[952, 319], [1126, 319], [1126, 382], [952, 382]], '"김해틀', 0.6891791224479675), ([[1138, 316], [1340, 316], [1340, 385], [1138, 385]], '아름답게', 0.9564188718795776), ([[1352, 316], [1506, 316], [1506, 382], [1352, 382]], '시민흘', 0.21049218836622702), ([[1518, 316], [1742, 316], [1742, 385], [1518, 385]], '행복하게"', 0.9996494662489984), ([[954, 385], [1035, 385], [1035, 466], [954, 466]], '김', 0.9998815094763955), ([[1319, 381], [1408, 381], [1408, 465], [1319, 465]], '해', 0.9999432571831335), ([[367, 481], [420, 481], [420, 539], [367, 539]], '우', 0.9999682905803553), ([[433, 480], [676, 480], [676, 540], [433, 540]], '621-010/', 0.8502205921099988), ([[713, 477], [832, 477], [832, 543], [713, 543]], '경남', 0.9630026380685481), ([[833, 477], [993, 477], [993, 543], [833, 543]], '김해시', 0.9941473818466108), ([[1010, 477], [1168, 477], [1168, 543], [1010, 543]], '부원동', 0.9962835228060669), ([[1180, 480], [1284, 480], [1284, 540], [1180, 5

## 인식 영역만 실행
recognizer 파라미터 사용

In [1]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 1개)
file_paths = file_paths[:1]
file_count = 1


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], recognizer=False)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    # 문자 인식 결과 출력
    print("")
    print("<OCR 인식 결과>")
    print(result)
    # 인식한 문자 출력
    for i in result:
        print(i[1])
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        # 라벨링 데이터의 문자 값만 출력
        print("")
        print("<라벨링 데이터>")
        # 인식한 문자 출력
        for k in json_test:
            print(k['annotation.text'])
            
        print("경과 시간 : {:.2f}초".format(elapsed_time))     
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))

C:\Users\edongmin\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중


AttributeError: 'Reader' object has no attribute 'recognizer'

버전에 따른 문제인지 recognizer 파라미터로 사용하면 오류가 발생한다.

## 인식 영역만 실행

readtext -> detect 로 변경

In [11]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 1개)
file_paths = file_paths[:1]
file_count = 1


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.detect(folder_path_source+"/"+filename_split+".jpg")
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    
    # 문자 인식 결과 출력
    print("")
    print("<OCR 인식 결과>")
    print(result)
    # 좌표 값 출력
    for i in result:
        print(i[0])
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        # 라벨링 데이터의 좌표 값만 출력
        print("")
        print("<라벨링 데이터>")
        # bbox 좌표 값 출력
        for k in json_test:
            print(k['annotation.bbox'])
    
        # 경과 시간 출력
        print("")
        print("경과 시간 : {:.2f}초".format(elapsed_time))
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중

<OCR 인식 결과>
([[[952, 1742, 316, 385], [954, 1035, 385, 466], [1319, 1408, 381, 465], [366, 676, 480, 540], [713, 1426, 477, 543], [1463, 2314, 477, 543], [361, 621, 550, 618], [682, 1066, 548, 621], [1130, 1537, 550, 618], [1571, 1778, 552, 618], [1846, 2003, 552, 618], [2037, 2250, 552, 618], [347, 818, 630, 696], [1370, 1541, 634, 686], [1593, 2325, 629, 697], [353, 560, 700, 768], [600, 859, 700, 765], [953, 1108, 698, 784], [1481, 1652, 720, 772], [2187, 2231, 727, 771], [353, 560, 774, 840], [597, 743, 777, 837], [867, 1120, 770, 859], [1477, 1651, 794, 854], [347, 549, 858, 921], [1480, 1653, 880, 940], [1956, 2166, 804, 950], [353, 537, 947, 1015], [572, 1348, 947, 1015], [1477, 1659, 972, 1037], [577, 976, 1038, 1106], [1971, 2147, 1030, 1090], [1477, 1717, 1060, 1129], [1753, 1888, 1067, 1122], [2114, 2251, 1084, 1163], [352, 544, 1134, 1210], [1476, 1660, 1154, 1223], [1677, 1990, 1136, 1265], [2350, 2520, 1193, 1266], [1500, 1630, 12

cpu 환경으로 실행

In [3]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 1개)
file_paths = file_paths[:1]
file_count = 1


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'])

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.detect(folder_path_source+"/"+filename_split+".jpg")
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    
    # 문자 인식 결과 출력
    print("")
    print("<OCR 인식 결과>")
    print(result)
    # 좌표 값 출력
    for i in result:
        print(i[0])
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        # 라벨링 데이터의 좌표 값만 출력
        print("")
        print("<라벨링 데이터>")
        # bbox 좌표 값 출력
        for k in json_test:
            print(k['annotation.bbox'])
    
        # 경과 시간 출력
        print("")
        print("경과 시간 : {:.2f}초".format(elapsed_time))
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중

<OCR 인식 결과>
([[[952, 1742, 316, 385], [954, 1035, 385, 466], [1319, 1408, 381, 465], [366, 676, 480, 540], [713, 1426, 477, 543], [1463, 2314, 477, 543], [361, 621, 550, 618], [682, 1066, 548, 621], [1130, 1537, 550, 618], [1571, 1778, 552, 618], [1846, 2003, 552, 618], [2037, 2250, 552, 618], [347, 818, 630, 696], [1370, 1541, 634, 686], [1593, 2325, 629, 697], [353, 560, 700, 768], [600, 859, 700, 765], [953, 1108, 698, 784], [1481, 1652, 720, 772], [2187, 2231, 727, 771], [353, 560, 774, 840], [597, 743, 777, 837], [867, 1120, 770, 859], [1477, 1651, 794, 854], [347, 549, 858, 921], [1480, 1653, 880, 940], [1956, 2166, 804, 950], [353, 537, 947, 1015], [572, 1348, 947, 1015], [1477, 1659, 972, 1037], [577, 976, 1038, 1106], [1971, 2147, 1030, 1090], [1477, 1717, 1060, 1129], [1753, 1888, 1067, 1122], [2114, 2251, 1084, 1163], [352, 544, 1134, 1210], [1476, 1660, 1154, 1223], [1677, 1990, 1136, 1265], [2350, 2520, 1193, 1266], [1500, 1630, 12

## 문자 값만 출력

readtext -> recognize로 변경

In [10]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 1개)
file_paths = file_paths[:1]
file_count = 1


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.recognize(folder_path_source+"/"+filename_split+".jpg")
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    # 문자 인식 결과 출력
    print("")
    print("<OCR 인식 결과>")
    print(result)
    # 인식한 문자 출력
    for i in result:
        print(i[1])
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        # 라벨링 데이터의 문자 값만 출력
        print("")
        print("<라벨링 데이터>")
        # 인식한 문자 출력
        for k in json_test:
            print(k['annotation.text'])
            
        print("경과 시간 : {:.2f}초".format(elapsed_time))     
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중

<OCR 인식 결과>
([[0, 0], [2755, 0], [2755, 3553], [0, 3553]], '끈', 0.07173732262941712)
끈

<라벨링 데이터>
"김해를
아름답게
시민을
행복하게"
김
해
시
우
621-010/
경남
김해시
부원동
623
번지/
전화(055)330-3363
/전송330-3689
허가민원과
과장
류보현
공장설립담당주사
감경석
담당자
이창수
http://gimhae.gc.kr
E-mail
:
lcs@mail.kimhae.kyongnam.kr
문서번호
허가55142 -
4043
보존기간
시
장
시행일자
2001.
5.
4
(
년)
공개여부
4
(경
유)
부시장
수
신
부산시
사상구
감전동
159-5번지
실
장
동일중공업(주)대표
담당주사
과
장
전결
참
조
기
안
심사자
임삼성
목
공장설립
변경승인
통보
1.
귀하께서
제출한
공장설립변경승인신청에
대하여
공업배치
및
공장설립에
관한법률
제13조
제3항의
규정에
의거
붙임과
같이
승인서를
교부하오니,
조건사항
이행에
만전을
기하시기
바랍니다.
○
공장설립
변경승인
내용
회사명
소재지
대표자
업
종
(분류번호)
한림.안하
선박구성부분품
당초
영광산업사
1110-3
제조업
(35114)
승인
외1필지
변경
동일중공업
한림.안하
1110-3
승인
(주)
외1필지
제조업
(29169)
변경사유
:
회사명(대표자)
업종,
건축면적
변경
2.
공장설립승인
및
농지전용허가에
따른
지방세법
제161조
및
동법시행령
제124조의
규정에
의한
면허세(공장
:
8,000원,
농지
18,000원)를
납부하여야
합니다.
3.
건축물
사용승인을
얻고
기계.장치의
설치를
완료한
후
2개월이내에
공장
설립완료신고를
하여야
합니다.
이를
위반하는
경우에는
200만원이하의
과태료처분을
받게되오니
이점유념하시기
바랍니다.
4.
우리시는
공장을
설립하는
업체에
대하여
특별한
사유가
없는
한
본사를
우리시에
소재하도록
유도하고
있습니다

무슨 이유인지 인식이 잘 되지 않는다.

### 결론

1. 인식 영역만 추출하려면 readtext 대신 detect를 이용
2. 속도 비교하면 거의 차이가 없다. / cpu <-> gpu 환경의 실행 속도 차이가 크다.